# Feature Translation Service SWORD River Data Demo

This notebook shows a few example requests to the Feature Translation Service (FTS) Application Programming Interface (API).

## Required Dependencies

There are three python dependencies that must be available to the python kernel running this notebook. 

- plotly (https://pypi.org/project/plotly/)
- kaleido (https://pypi.org/project/kaleido/)
- geojson (https://pypi.org/project/geojson/)

The next cell installs them when the cell is run.

In [ ]:
!pip install plotly geojson kaleido

In [ ]:
import requests
import json
import geojson
import pprint
import plotly.graph_objects as go

from IPython.display import JSON, Image

def response_to_FeatureCollection(response):
    """
    This function will return a geojson.FeatureCollection representation of the features found
    in the provided response.
    Parameters
    ----------
    response : requests.Response
        Response object returned from a GET request on the FTS rivers endpoint.
    Returns
    -------
    geojson.FeatureCollection
        FeatureCollection containing all features extracted from the response.
    """
    featureList = []
    for reach_id, reach_json in response.json()['results'].items():
        reach_feature = geojson.loads(json.dumps(reach_json['geojson']))
        reach_feature['properties']={k:v for k,v in reach_json.items() if k not in ['geojson', 'geometry']}
        featureList.append(reach_feature)
    featureCollection = geojson.FeatureCollection(featureList)
    return featureCollection

def estimate_center_of_FeatureCollection(featureCollection):
    """
    This function does a very simplistic estimation of the center of all features in the given FeatureCollection.
    Parameters
    ----------
    featureCollection : geojson.FeatureCollection
        Estimate the center lon, lat of this FeatureCollection.
    Returns
    -------
    tuple(float, float)
        Estimated center longitude, center latitude
    """
    lats = [xy[1] for feature in featureCollection['features'] for xy in feature['coordinates']]
    lons = [xy[0] for feature in featureCollection['features'] for xy in feature['coordinates']]

    center_lat = (min(lats) + max(lats)) / 2
    center_lon = (min(lons) + max(lons)) / 2
    
    return center_lon, center_lat

# Plot a single River Reach

In [ ]:
response = requests.get("https://fts.podaac.earthdata.nasa.gov/rivers/reach/74230900271")

featureCollection = response_to_FeatureCollection(response)

pprint.pprint(response.json(), compact=True, width=60, depth=2)

In [ ]:
fig = go.Figure(
    go.Scattermapbox(),
    layout = {
        'mapbox': {
            'style': "stamen-terrain",
            'zoom': 12
        },
        'margin': {'l':0, 'r':0, 'b':0, 't':0},
    }
)

fig.update_mapboxes(layers=[
    {
        'source': feature, 
        'type': "line", 
        'line': {'width': 5},
        'below': "traces", 
        'color': "royalblue"
    } for feature in featureCollection.features])

center_lon, center_lat = estimate_center_of_FeatureCollection(featureCollection)

fig.update_layout(
    mapbox = {
        'center': { 'lon':  center_lon, 'lat': center_lat}
    }
)

Image(fig.to_image(format="png", width=600, height=600))

In [ ]:
fig.show()

# Plot m

In [ ]:
fig = go.Figure(
    go.Scattermapbox(),
    layout = {
        'mapbox': {
            'style': "stamen-terrain",
            'zoom': 8
        },
        'margin': {'l':0, 'r':0, 'b':0, 't':0},
    }
)

fig.update_mapboxes(layers=[
    {
        'source': feature, 
        'type': "line", 
        'line': {'width': 5},
        'below': "traces", 
        'color': "royalblue"
    } for feature in featureCollection.features])

center_lon, center_lat = estimate_center_of_FeatureCollection(featureCollection)

fig.update_layout(
    mapbox = {
        'center': { 'lon':  center_lon, 'lat': center_lat}
    }
)

Image(fig.to_image(format="png", width=600, height=600))

In [ ]:
fig.show()

In [ ]:
response = requests.get("https://fts.podaac.earthdata.nasa.gov/rivers/node/74230900100")

featureCollection = response_to_FeatureCollection(response)

pprint.pprint(response.json(), compact=True, width=60, depth=2)

In [ ]:

fig = go.Figure(
    go.Scattermapbox(),
    layout = {
        'mapbox': {
            'style': "stamen-terrain",
            'zoom': 11
        },
        'margin': {'l':0, 'r':0, 'b':0, 't':0},
    }
)

fig.update_mapboxes(layers=[
    {
        'source': feature, 
        'type': "circle", 
        'circle': {'radius': 3},
        'below': "traces", 
        'color': "royalblue"
    } for feature in featureCollection.features])

lats = [feature['coordinates'][1] for feature in featureCollection['features']]
lons = [feature['coordinates'][0] for feature in featureCollection['features']]

center_lat = (min(lats) + max(lats)) / 2
center_lon = (min(lons) + max(lons)) / 2

fig.update_layout(
    mapbox = {
        'center': { 'lon':  center_lon, 'lat': center_lat}
    }
)

Image(fig.to_image(format="png", width=600, height=600))

In [ ]:
fig.show()